# TO DO: CARGAR LOS OUTPUTS DE LOS DOS MODELOS

In [1]:
import pandas as pd

cnn_probs_train = pd.read_csv('/kaggle/input/cnn-results/predicts_vs_real_TRAIN.csv', index_col=0)
cnn_probs_test = pd.read_csv('/kaggle/input/cnn-results/predicts_vs_real_TEST.csv', index_col=0)
cnn_columns = ['cnn_prob_0', 'cnn_prob_1', 'cnn_prob_2', 'cnn_prob_3', 'cnn_prob_4', 'cnn_prob_5', 'tag']

In [2]:
tree_probs_train = pd.read_csv('/kaggle/input/tree-results/predicts_vs_real_TRAIN.csv', index_col=0)
print('Train checker:', all(tree_probs_train.tag.values == cnn_probs_train.tag.values))
tree_probs_train.drop(columns = 'tag', inplace=True)

tree_probs_test = pd.read_csv('/kaggle/input/tree-results/predicts_vs_real_TEST.csv', index_col=0)
print('Test checker:', all(tree_probs_test.tag.values == cnn_probs_test.tag.values))
tree_probs_test.drop(columns = 'tag', inplace=True)
tree_columns = ['tree_prob_0', 'tree_prob_1', 'tree_prob_2', 'tree_prob_3', 'tree_prob_4', 'tree_prob_5']

Train checker: True
Test checker: True


# TO DO: CREAR DOS CONJUNTOS TRAIN Y TEST CON SUS X E Y

In [3]:
cnn_probs_train.columns = cnn_probs_test.columns = cnn_columns
tree_probs_train.columns = tree_probs_test.columns = tree_columns

x_train = pd.concat([tree_probs_train, cnn_probs_train], axis=1)
y_train = x_train.tag
x_train.drop(columns='tag', inplace=True)

x_test = pd.concat([tree_probs_test, cnn_probs_test], axis=1)
y_test = x_test.tag
x_test.drop(columns='tag', inplace=True)

# TO DO: ENTRENAR UN MODELO DE REGRESION QUE HAGA DE ENSEMBLE ENTRE LOS DOS DE PARTIDA

In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

lr_model = LogisticRegression(max_iter=5000)

parameters = {}
parameters['C'] = [10e-3, 10e-2, 10e-1, 1, 10, 100, 1000]
parameters['class_weight'] = [None, 'balanced']

GS = GridSearchCV(lr_model, parameters)
GS.fit(x_train, y_train)

lr_model = GS.best_estimator_

pred_probs_test = lr_model.predict_proba(x_test)
y_test_pred = np.argmax(pred_probs_test, axis=1)
report = classification_report(y_test, y_test_pred, output_dict=True)
pd.DataFrame(report).T

,precision,recall,f1-score,support
0,0.651723,0.795869,0.716619,14138.00000
1,0.820488,0.247790,0.380629,3394.00000
2,0.355004,0.285762,0.316642,11905.00000
3,0.749964,0.702997,0.725721,7441.00000
4,0.538423,0.607710,0.570972,10584.00000
5,0.601612,0.632568,0.616702,12982.00000
accuracy,0.585170,0.585170,0.585170,0.58517
macro avg,0.619536,0.545449,0.554548,60444.00000
weighted avg,0.584250,0.585170,0.573131,60444.00000


Mejoramos **~1%** con respecto a la media aritmetica de las predicciones